In [69]:
import sklearn as sk
import numpy as np
import pandas as pd
import rasterio, os, napari
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans

In [70]:
with open(r"data.npy") as file:
    data = np.load(file)

In [71]:
with open("labels.npy", "rb") as file:
    labels = np.load(file)

### Data Cleaning

In [72]:
data.shape

(87477, 9)

In [73]:
data.shape

(87477, 9)

In [74]:
data.shape
filtered = [[point[0],point[1]] for point in zip(data, labels) if np.sum(np.abs(point[0])) < 255*9]

In [75]:
len(filtered)

85132

In [76]:
data, label = zip(*filtered)

In [77]:
 X_train, X_test, y_train, y_test = train_test_split(data,label, test_size=0.2)

In [78]:
a = pd.Series(y_train).value_counts()
b = pd.Series(y_test).value_counts()

print(a)
print("\n")
print(b)

Not Ice    47514
Ice        20591
dtype: int64


Not Ice    11955
Ice         5072
dtype: int64


### Random Forest

In [138]:
fclf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

In [139]:
X_train

[array([1.50212854e-01, 1.58476904e-01, 1.72138572e-01, 1.70211524e-01,
        1.52199373e-01, 3.04382446e+02, 3.04471802e+02, 1.27948135e-01,
        1.72454074e-01]),
 array([1.00785300e-01, 7.99760669e-02, 7.12981597e-02, 6.17248453e-02,
        4.01076376e-02, 2.81891327e+02, 2.81529602e+02, 3.19386423e-02,
        7.31761903e-02]),
 array([1.78966954e-01, 1.90967485e-01, 2.11786315e-01, 2.22162679e-01,
        2.21056372e-01, 3.06278961e+02, 3.05788544e+02, 1.81247801e-01,
        2.22311795e-01]),
 array([2.21037090e-01, 2.27815405e-01, 2.33012050e-01, 2.27259800e-01,
        1.79395899e-01, 2.93931946e+02, 2.93702423e+02, 1.60147205e-01,
        2.54044622e-01]),
 array([3.42951924e-01, 3.29193324e-01, 3.16716373e-01, 2.90891409e-01,
        3.80754173e-02, 2.70271423e+02, 2.70319733e+02, 2.98318118e-02,
        3.02732676e-01]),
 array([4.55713451e-01, 4.51957971e-01, 4.74158078e-01, 4.30367768e-01,
        5.98295219e-02, 2.81891327e+02, 2.82490570e+02, 4.75070961e-02,
      

In [140]:
fclf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10)

In [141]:
fclf.score(X_test, y_test)

0.7394138720855112

In [142]:
fclf.predict(X_test)


array(['Not Ice', 'Not Ice', 'Not Ice', ..., 'Not Ice', 'Not Ice',
       'Not Ice'], dtype='<U7')

In [143]:
fclf.predict(X_test)

array(['Not Ice', 'Not Ice', 'Not Ice', ..., 'Not Ice', 'Not Ice',
       'Not Ice'], dtype='<U7')

In [144]:
sk.metrics.f1_score(y_test,fclf.predict(X_test), pos_label='Not Ice')

0.830823197468258

### Decision Tree

In [186]:
dtr = lm.LogisticRegression(random_state=0).fit(X_train, y_train)

C:\Users\marke\miniconda3\envs\capstone\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [187]:
dtr.score(X_test, y_test)

0.6924884007752393

In [188]:
sk.metrics.f1_score(y_test,dtr.predict(X_test), pos_label='Not Ice')

0.8137847642079806

### SVM

In [184]:
sclf = SVC(gamma='auto')

In [185]:
sclf.fit(X_train,y_train)

KeyboardInterrupt: 

In [45]:
sclf.score(X_test, y_test)

0.7407261247040252

In [46]:
sk.metrics.f1_score(y_test,sclf.predict(X_test), pos_label='Not Ice')

0.8510541827250055

### Dimentionality Reduction

In [20]:
cclf = KMeans()

In [21]:
cclf.fit(X_train)

KMeans()

# Display prediction

In [173]:
def runNapari(img_path, full_img_path,rgb_flag):
    with rasterio.open(img_path) as src:
        img = src.read()
    
    with rasterio.open(full_img_path) as src:
        img_full = src.read()

    viewer = napari.view_image(img, show=False)
    return viewer, img, img_full


In [174]:
def loadAndTest(full_directory,rgb_directory, label_directory, trained_model):
    
    viewer,ndsi,full = runNapari(rgb_directory,full_directory)
    e_dims = full.shape
    print(e_dims)
    testbals = np.rollaxis(full.reshape(9,e_dims[1]*e_dims[2]),0,2)
    print(testbals)
    prediction = trained_model.predict(testbals)

    prediction = prediction.reshape(e_dims[1],e_dims[2])
    # np.count_nonzero(testbals.reshape(9,145,363) != full)
    display_dims = prediction.shape
    img_coordinates = np.array(list(zip(np.arange(0,display_dims[0]),np.arange(0,display_dims[1]))))
    
    ice_indexes = np.where(prediction == "Ice")
    non_ice_indexes = np.where(prediction == "Not Ice")
    ice_coordinates = list(zip(ice_indexes[0],ice_indexes[1]))
    non_ice_coordinates = list(zip(non_ice_indexes[0], non_ice_indexes[1]))
    
    viewer = napari.view_image(ndsi)
    viewer.add_points(ice_coordinates,face_color="red",edge_color ="red",size=1, name="Ice")
    viewer.add_points(non_ice_coordinates,face_color="blue",edge_color ="blue",size=1, opacity=0,name="Not Ice")
    
    viewer.screenshot(label_directory)

In [181]:
directory_path = r"C:\Users\marke\Documents\DSC180A-Q1\testing_data\full"
files = os.listdir(directory_path)
files

['Aletsch_glacier_1999-07-02.tif',
 'Aletsch_glacier_1999-07-18.tif',
 'Aletsch_glacier_1999-08-10.tif',
 'Aletsch_glacier_1999-08-19.tif',
 'Aletsch_glacier_1999-09-11.tif',
 'Aletsch_glacier_1999-10-06.tif']

In [182]:
# test_img = "Engilchek_glacier_2001-03-24.tif"
# test_img = r"Engilchek_glacier_1999-08-26.tif"
test_img = r"Engilchek_glacier_1999-07-09"

In [189]:
for file in files:
    file_name = file.split(".")[0]
    full_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\testing_data\full",file)
    rgb_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\testing_data\rgb",file)
    label_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\labels",file_name+ r".png")
    loadAndTest(full_directory, rgb_directory, label_directory, fclf)
    print(full_directory)

(9, 376, 261)
[[3.48259717e-01 3.91778439e-01 3.71014774e-01 ... 2.71036377e+02
  3.00032273e-02 6.82049632e-01]
 [3.48259717e-01 3.91778439e-01 3.71014774e-01 ... 2.71036377e+02
  2.80065741e-02 6.82049632e-01]
 [3.48259717e-01 3.91778439e-01 3.71014774e-01 ... 2.71747742e+02
  2.40132697e-02 6.82049632e-01]
 ...
 [8.21517333e-02 6.68733194e-02 4.82048839e-02 ... 2.96272552e+02
  5.39630502e-02 1.12722695e-01]
 [8.21517333e-02 6.36879727e-02 4.36795093e-02 ... 2.96272552e+02
  4.19831388e-02 1.09985545e-01]
 [7.93207958e-02 7.00586587e-02 4.82048839e-02 ... 2.96272552e+02
  4.39797901e-02 1.12722695e-01]]


C:\Users\marke\Documents\DSC180A-Q1\testing_data\full\Aletsch_glacier_1999-07-02.tif
(9, 376, 261)
[[3.5484788e-01 3.9922559e-01 3.7805209e-01 ... 2.6776797e+02
  3.2607675e-02 6.5312624e-01]
 [3.5484788e-01 3.9922559e-01 3.7805209e-01 ... 2.6813678e+02
  4.6849538e-02 5.8340204e-01]
 [3.5484788e-01 3.9922559e-01 3.7805209e-01 ... 2.6813678e+02
  5.7022296e-02 6.3360345e-01]
 ...
 [1.1255076e-01 9.8980457e-02 8.6008847e-02 ... 2.9513708e+02
  8.5506022e-02 1.4274517e-01]
 [9.3801573e-02 7.6259203e-02 5.2193314e-02 ... 2.9513708e+02
  3.0573124e-02 1.2043343e-01]
 [9.5243819e-02 7.4636258e-02 5.0656244e-02 ... 2.9485153e+02
  3.0573124e-02 1.1764447e-01]]


KeyboardInterrupt: 

In [163]:
loadAndTest(full_directory, rgb_directory, label_directory, fclf)

(9, 176, 419)
[[3.4324351e-01 3.8616636e-01 3.6570123e-01 ... 2.7419736e+02
  2.0078249e-01 4.3753985e-01]
 [3.4324351e-01 3.8616636e-01 3.6570123e-01 ... 2.7522888e+02
  3.5475679e-02 6.5336603e-01]
 [2.8185999e-01 2.8098589e-01 3.0325356e-01 ... 2.8532312e+02
  1.4371228e-01 2.8646150e-01]
 ...
 [1.5211760e-01 1.4597812e-01 1.5159494e-01 ... 2.9137015e+02
  9.0577953e-02 1.5157011e-01]
 [1.8420443e-01 1.9464371e-01 2.1552946e-01 ... 2.9254208e+02
  1.9684662e-01 2.3520276e-01]
 [1.9117983e-01 2.0877244e-01 2.3931904e-01 ... 2.9485153e+02
  2.3030157e-01 2.4869190e-01]]


### Scratch

In [271]:
sclf

SVC(gamma='auto')

In [169]:
full,ndsi = runNapari(test_img,full_directory,ndsi_directory)

In [247]:
e_dims = full.shape
testbals = np.rollaxis(full.reshape(9,e_dims[1]*e_dims[2]),0,2)
prediction = fclf.predict(testbals)

prediction = prediction.reshape(e_dims[1],e_dims[2])
# np.count_nonzero(testbals.reshape(9,145,363) != full)
display_dims = prediction.shape
display_dims

(145, 363)

In [239]:
img_coordinates = np.array(list(zip(np.arange(0,display_dims[1]),np.arange(0,display_dims[2]))))

In [250]:
ice_indexes = np.where(prediction == "Ice")
non_ice_indexes = np.where(prediction == "Not Ice")

In [251]:
ice_coordinates = list(zip(ice_indexes[0],ice_indexes[1]))
non_ice_coordinates = list(zip(non_ice_indexes[0], non_ice_indexes[1]))

In [253]:
viewer = napari.view_image(ndsi)

In [255]:
viewer.add_points(ice_coordinates,face_color="red",edge_color ="red",size=1, name="Ice")
viewer.add_points(non_ice_coordinates,face_color="blue",edge_color ="blue",size=1, name="Not Ice")


<Points layer 'Not Ice' at 0x21aaee09520>